In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import statsmodels.api as sm
import scipy
from scipy.stats import norm
from scipy.stats.mstats import gmean

from statsmodels.nonparametric.smoothers_lowess import lowess

from numpy.lib.stride_tricks import sliding_window_view
import functions_mem as fm

from datetime import datetime
from pathlib import Path

In [2]:
import importlib
import functions_mem
importlib.reload(functions_mem)



<module 'functions_mem' from '/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py'>

In [3]:
dir(functions_mem)


['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'add_sea',
 'baseline_thresholds',
 'find_best_season_and_delta',
 'gmean',
 'lowess',
 'mem_epidemic_period',
 'mem_epidemic_period_old',
 'norm',
 'np',
 'pd',
 'sliding_window_view',
 'smooth_lowess',
 'summary_is_valid']

# Load data

In [5]:
df = pd.read_parquet('/opt/storage/shared/aesop/aesop_shared/ensamble_modelling/aesop_2026_01_21_mun.parquet')
    
    #"/Users/julianeoliveira/Downloads/aesop_2026_01_09_mun.parquet")

In [6]:
df.head()

,co_ibge,epiyear,epiweek,atend_totais,atend_ivas,atend_arbov,atend_diarreicas,atend_0a11,year_week,epidemi_cal_start,...,sinal_ens_consec_arbov_012,sinal_otc_consec_ivas_012,sinal_aesop_consec_ivas_012,ano_right,epiweek_right,EWS_ISF_right,EWS_LOF_right,EWS_OCSVM_right,EWS_COPOD_right,EWS_Rt_right
0,110001,2017,1,428,9,0,0,103,2017-01,2017-01-01,...,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None
1,110001,2017,2,741,16,1,2,167,2017-02,2017-01-08,...,1.0,0.0,0.0,NaN,NaN,None,None,None,None,None
2,110001,2017,3,576,15,0,0,131,2017-03,2017-01-15,...,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None
3,110001,2017,4,701,17,0,1,92,2017-04,2017-01-22,...,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None
4,110001,2017,5,703,11,0,1,88,2017-05,2017-01-29,...,0.0,0.0,0.0,NaN,NaN,None,None,None,None,None


In [7]:
lst = ['co_uf','nm_uf','nm_municipio','co_ibge', 'epiyear', 'epiweek','year_week', 'atend_totais', 'atend_ivas','ra_atend_ivas','ra_atend_ivas_ma',]

df = df[lst]

# Start run method

### Test the code for one city (this part of the code will is commented when running the python file)

In [8]:
res = fm.find_best_season_and_delta(
    df,
    110001,
    delta_start=0.02,
    delta_max=0.04,
    delta_step=0.001,
    k_start_min=10,
    r_max=20,
)

In [9]:
print(res)


   epiyear  r_j_estr  k_start  k_end  co_ibge  season_def  delta_used  co_uf  \
0     2017        27        2     28   110001  season_w39       0.022     11   
1     2018        13       35     47   110001  season_w39       0.022     11   
2     2019        15       36     50   110001  season_w39       0.022     11   
3     2020        12       41     52   110001  season_w39       0.022     11   
4     2021        18       17     34   110001  season_w39       0.022     11   
5     2022        15        2     16   110001  season_w39       0.022     11   
6     2023         8       11     18   110001  season_w39       0.022     11   
7     2024         9       28     36   110001  season_w39       0.022     11   
8     2025        14       31     44   110001  season_w39       0.022     11   
9     2026         8       10     17   110001  season_w39       0.022     11   

      nm_uf  
0  Rondônia  
1  Rondônia  
2  Rondônia  
3  Rondônia  
4  Rondônia  
5  Rondônia  
6  Rondônia  
7  Rond

### Run for all cities

In [ ]:
results = []

for city in df.co_ibge.unique():
    print(f"Processing {city}...")
    res = fm.find_best_season_and_delta(df, city)
    if res is not None:
        results.append(res)

final_selected = pd.concat(results, ignore_index=True)


Processing 110001...
Processing 110002...
Processing 110003...
Processing 110004...
Processing 110005...
Processing 110006...
Processing 110007...


/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing 110008...
Processing 110009...
Processing 110010...
Processing 110011...
Processing 110012...
Processing 110013...
Processing 110014...
Processing 110015...
Processing 110018...
Processing 110020...
Processing 110025...
Processing 110026...
Processing 110028...
Processing 110029...
Processing 110030...
Processing 110032...
Processing 110033...
Processing 110034...
Processing 110037...
Processing 110040...
Processing 110045...
Processing 110050...
Processing 110060...
Processing 110070...
Processing 110080...
Processing 110090...
Processing 110092...


/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:2

Processing 110094...
Processing 110100...


...

Processing 521405...
Processing 521410...
Processing 521440...
Processing 521450...
Processing 521460...
Processing 521470...
Processing 521480...
Processing 521483...
Processing 521486...
Processing 521487...
Processing 521490...
Processing 521500...
Processing 521520...
Processing 521523...
Processing 521525...
Processing 521530...
Processing 521540...
Processing 521550...
Processing 521560...


/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing 521565...
Processing 521570...
Processing 521580...
Processing 521590...
Processing 521600...


/home/juliane.oliveira/workspace/IDEA_model/tmp/functions_mem.py:275: RuntimeWarning: invalid value encountered in divide
  p_j_r = t_j_r / tS
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Processing 521630...
Processing 521640...
Processing 521645...
Processing 521680...
Processing 521690...
Processing 521710...
Processing 521720...


# Save output to parquet


In [ ]:
out_dir = Path("/opt/storage/shared/aesop/aesop_shared/ensamble_modelling")
#/Users/julianeoliveira/Downloads")

fname = f"def_sea_MEM_out_{datetime.now():%d_%m_%Y}.parquet"

final_selected.to_parquet(out_dir / fname)


In [ ]:
set_bad =  set(df.co_ibge.unique()) - set(final_selected.co_ibge.unique())
dta = pd.DataFrame({"co_ibge": sorted(set_bad)})

fname2 = f"set_bad_cities_{datetime.now():%d_%m_%Y}.csv"

dta.to_csv(out_dir / fname2, index=False)

In [ ]:
#final_selected.to_parquet('/Users/julianeoliveira/Downloads/def_sea_MEM_23_01_2026.parquet')

In [ ]:
final_selected.co_ibge.nunique()